<a href="https://colab.research.google.com/github/PritamGoyal/Automated-mcq-s-generator/blob/main/automated_mcqs_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install transformers
!pip install torch
!pip install tensorflow
!pip install keybert
!pip install -U nltk
!pip install -U pywsd
!pip install gensim
!pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-ee76jwug
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-ee76jwug
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done


In [22]:
%config Completer.use_jedi=False


In [23]:
!pip install nltk

In [24]:
import nltk

In [25]:
from transformers import pipeline
from transformers import BartForConditionalGeneration, BartTokenizer
from keybert import KeyBERT


In [26]:
import tensorflow as tf
import torch
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import random
import copy
import sys


In [27]:
from nltk import wsd
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from spacy.cli import download
from spacy import load
import warnings
from nltk.tokenize import sent_tokenize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [29]:
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')
nlp = load('en_core_web_sm')

! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet # temp fix for lookup error.

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to /root/nltk_data...
[nltk_data]   Package wordnet2022 is already up-to-date!


cp: cannot stat '/usr/share/nltk_data/corpora/wordnet2022': No such file or directory


In [30]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
import requests
import json
import re
import random
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [32]:
model= BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer= BartTokenizer.from_pretrained("facebook/bart-large-cnn")

In [33]:

def summarize(text, maxSummarylength=2000):
  inputs= tokenizer.encode("summarize: "+text,
                           return_tensors="pt",
                           max_length=1024, truncation=True)
  summary_ids= model.generate(inputs, max_length=maxSummarylength,
                              min_length= int(maxSummarylength/5),
                              length_penalty=10.0,
                              num_beams= 4, early_stopping= True)
  summary= tokenizer.decode(summary_ids[0], skip_special_tokens= True)
  return summary

In [34]:
def split_text_into_pieces(text, max_tokens=3000, overlapPercent=10):
  tokens=tokenizer.tokenize(text)
  overlap_tokens=int(max_tokens * overlapPercent/100)
  pieces=[tokens[i:i + max_tokens]
          for i in range(0, len(tokens),
                         max_tokens - overlap_tokens)]
  text_pieces= [tokenizer.decode(
      tokenizer.convert_tokens_to_ids(piece),
      skip_special_tokens= True) for piece in pieces]

  return text_pieces

In [35]:
def recursive_summarize(text, max_length=2000, recursionLevel=0):
    recursionLevel=recursionLevel+1
    print("######### Recursion level: ",
          recursionLevel,"\n\n######### ")
    tokens = tokenizer.tokenize(text)
    expectedCountOfChunks = len(tokens)/max_length
    max_length=int(len(tokens)/expectedCountOfChunks)+2

    # Break the text into pieces of max_length
    pieces = split_text_into_pieces(text, max_tokens=max_length)

    print("Number of pieces: ", len(pieces))
    # Summarize each piece
    summaries=[]
    k=0
    for k in range(0, len(pieces)):
        piece=pieces[k]
        print("****************************************************")
        print("Piece:",(k+1)," out of ", len(pieces), "pieces")
        print(piece, "\n")
        summary =summarize(piece, maxSummarylength=max_length/3*2)
        print("SUMNMARY: ", summary)
        summaries.append(summary)
        print("****************************************************")

    concatenated_summary = ' '.join(summaries)

    tokens = tokenizer.tokenize(concatenated_summary)

    if len(tokens) > max_length:
        # If the concatenated_summary is too long, repeat the process
        print("############# GOING RECURSIVE ##############")
        return recursive_summarize(concatenated_summary,
                                   max_length=max_length,
                                   recursionLevel=recursionLevel)
    else:
      # Concatenate the summaries and summarize again
        final_summary=concatenated_summary
        if len(pieces)>1:
            final_summary = summarize(concatenated_summary,
                                  maxSummarylength=max_length)
        return final_summary

In [36]:
def read_text_from_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print("File not found. Please ensure the path is correct.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [37]:
model_2= KeyBERT(model="distilbert-base-nli-mean-tokens")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [38]:
#more relevant
%%time
def keyword_extract(article):
  keywords=model_2.extract_keywords(article,
                        top_n=200,
                        keyphrase_ngram_range=(1, 1), #can also generate phrases of more than 1 word (1,2), (1,3)
                        stop_words="english")
  return (keywords)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 10.5 µs


In [39]:
# Automatic POS Tagging & Lesk with spacy

POS_MAP = {
    'VERB': wn.VERB,
    'NOUN': wn.NOUN,
    'PROPN': wn.NOUN
}


def lesk(doc, word):
    found = False
    for token in doc:
        if token.text.lower() == word.lower():
            word = token
            found = True
            break
    if not found:
        #warning.warn(f'Word \"{word}\" does not appear in the document: {doc.text}.')
        return None
    pos = POS_MAP.get(word.pos_, False)
    if not pos:
        #warnings.warn(f'POS tag for {word.text} not found in wordnet. Falling back to default Lesk behaviour.')
        return None
    args = [c.text for c in doc], word.text
    kwargs = dict(pos=pos)
    return wsd.lesk(*args, **kwargs)

In [40]:
def corresponding_sentences(text,filtered_keywords):
  sentences=sent_tokenize(text)      #Sentence tokenization
  corr_sentences=dict()
  for keyword in filtered_keywords:
    for sentence in sentences:
      if keyword.lower() in sentence.lower():
        corr_sentences.update({keyword:sentence})  #if keyword is found in the sentence then the dictionary is updated with the corresponding key value pair
  return(corr_sentences)

In [41]:
def get_wordsense(sent, word):
    word = word.lower()

    if len(word.split()) > 0:
        word = word.replace(" ", "_")

    synsets = wn.synsets(word, 'n')

    if synsets:
        try:
            wup = max_similarity(sent, word, 'wup', pos='n')
            adapted_lesk_output = adapted_lesk(sent, word, pos='n')
            lowest_index = min(synsets.index(wup), synsets.index(adapted_lesk_output))
            return synsets[lowest_index]
        except (IndexError,ValueError):
            return None
    else:
        return None

In [42]:
def get_distractors_wordnet(syn, word):
    distractors = []
    word = word.lower()
    orig_word = word

    if len(word.split()) > 0:
        word = word.replace(" ", "_")

    hypernym = syn.hypernyms()

    if len(hypernym) == 0:
        return distractors

    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()

        if name == orig_word:
            continue

        name = name.replace("_", " ")
        name = " ".join(w.capitalize() for w in name.split())

        if name is not None and name not in distractors:
            distractors.append(name)

    return distractors


In [43]:
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word = word

    if len(word.split()) > 0:
        word = word.replace(" ", "_")

    distractor_list = []
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5" % (word, word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term']

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10" % (link, link)
        obj2 = requests.get(url2).json()

        for edge in obj2['edges']:
            word2 = edge['start']['label']

            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)

    return distractor_list

In [44]:
model_name_2 = "t5-small"
tokenizer_2 = T5Tokenizer.from_pretrained(model_name_2)
model_4 = T5ForConditionalGeneration.from_pretrained(model_name_2)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [45]:
def generate_mcq(keyword_sentences, key_distractor_list):
    index = 1
    for keyword, sentence in keyword_sentences.items():
        if keyword not in key_distractor_list:
            print(f"Skipping '{keyword}': No distractors found.")
            continue

        pattern = re.compile(re.escape(keyword), re.IGNORECASE)

        distractors = key_distractor_list[keyword]
        choices = [keyword.capitalize()] + distractors
        prompt = f"Here is a sentence with a blank: {sentence.replace(keyword, '_______')}. What is the missing word? Options: {', '.join(choices)}"

        input_ids = tokenizer_2(prompt, return_tensors="pt")
        output = model_4.generate(**input_ids)
        generated_answer = tokenizer_2.decode(output[0], skip_special_tokens=True)
        generated_answer = generated_answer.strip()

        masked_sentence = pattern.sub(generated_answer, sentence)
        print(f"{index}) {masked_sentence}")

        top4choices = choices[:4]
        random.shuffle(top4choices)
        optionchoices = ['a', 'b', 'c', 'd']
        for idx, choice in enumerate(top4choices):
            print(f"\t{optionchoices[idx]}) {choice}")
        print("\nMore options:", choices[4:], "\n")
        index += 1

In [46]:
choice=int(input("1 if you want to upload text file\n2 or any number if you want to enter text directly"))
if(choice==1):
  user_file_path = input("Please enter the path to your .txt file: ")
  article = read_text_from_file(user_file_path)
else:
  article=input("Enter the text: ")

summary= recursive_summarize(article)

original_keywords=keyword_extract(article)

required_keywords=[i[0] for i in original_keywords]

filtered_keywords=set()
for keyword in required_keywords:
  if(keyword.lower() in summary.lower()):
    filtered_keywords.add(keyword)

keyword_sentences=corresponding_sentences(summary, filtered_keywords)
key_distractor_list = {}

for keyword, sentence in keyword_sentences.items():
    wordsense = get_wordsense(sentence, keyword)
    if wordsense:
        distractors = get_distractors_wordnet(wordsense, keyword)
        if len(distractors) == 0:
            distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
    else:
        distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors

generate_mcq(keyword_sentences, key_distractor_list)

1 if you want to upload text file
2 or any number if you want to enter text directly1
Please enter the path to your .txt file: /content/chap_1.txt
######### Recursion level:  1 

######### 
Number of pieces:  2
****************************************************
Piece: 1  out of  2 pieces
Chapter 1
I am by birth a Genevese, and my family is one of the most distinguished of that republic. My ancestors had been for many years counsellors and syndics, and my father had filled several public situations with honour and reputation. He was respected by all who knew him for his integrity and indefatigable attention to public business. He passed his younger days perpetually occupied by the affairs of his country; a variety of circumstances had prevented his marrying early, nor was it until the decline of life that he became a husband and the father of a family.

As the circumstances of his marriage illustrate his character, I cannot refrain from relating them. One of his most intimate friends 

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


1) Two years after this event Caroline became his True.
	a) Helpmate
	b) Wife
	c) Consort
	d) Bigamist

More options: ['Husband', 'Monogamist', 'Newlywed', 'Polygamist'] 

2) In the tenth month her father died in her arms, leaving her an orphan and a True.
	a) Starveling
	b) Beggar
	c) Derelict

More options: [] 

Skipping 'bestow': No distractors found.
Skipping 'rapidly': No distractors found.
Skipping 'daughter': No distractors found.
3) My father was Truely grieved by his friend's retreat in these unfortunate circumstances.
	a) Bed
	b) Crater
	c) Deep
	d) Basin

More options: ['Hole', 'Lowland', 'Sinkhole', 'Trench', 'Trough', 'Valley'] 

4) There was a show of gratitude and worship in his attachment to my True, differing wholly from the doting fondness of age, for it was inspired by reverence for her virtues and a desire to be the means of, in some degree,ensing her for her wishes and her wishes.
	a) Afflatus
	b) Source
	c) Mother

More options: [] 

Skipping 'weeping': No distrac

Token indices sequence length is longer than the specified maximum sequence length for this model (1467 > 512). Running this sequence through the model will result in indexing errors


7) They consulted their _______ge priest, and the result was that Elizabeth Lavenza became the inmate ofge priest, and the result was that Elizabeth Lavenza became the inmate of my parents’ house.
	a) Villa
	b) gijón
	c) grandas
	d) avilés

More options: ['deva', 'ribadesella', 'aces', 'oviedo', 'santa cruz'] 

Skipping 'lost': No distractors found.
8) He came like a protecting spirit to the poor girl, who committed herself to his care; and after the interment of his True, Friend, Abator, Abjurer, Abomination, Abstainer he conducted her to Geneva and placed her under the protection of a relation.
	a) Abomination
	b) Friend
	c) Abator
	d) Abjurer

More options: ['Abstainer', 'Achiever', 'Acquaintance', 'Acquirer', 'Active', 'Actor', 'Adjudicator', 'Admirer', 'Adoptee', 'Adult', 'Adventurer', 'Adversary', 'Advisee', 'Advocate', 'Affiant', 'African', 'Agnostic', 'Amateur', 'Amerindian', 'Ancient', 'Anomaly', 'Anti-american', 'Anti', 'Applicant', 'Appointee', 'Appreciator', 'Apprehender', 

In [47]:
choice=int(input("1 if you want to upload text file\n2 or any number if you want to enter text directly"))
if(choice==1):
  user_file_path = input("Please enter the path to your .txt file: ")
  article = read_text_from_file(user_file_path)
else:
  article=input("Enter the text: ")

summary= recursive_summarize(article)

original_keywords=keyword_extract(article)

required_keywords=[i[0] for i in original_keywords]

filtered_keywords=set()
for keyword in required_keywords:
  if(keyword.lower() in summary.lower()):
    filtered_keywords.add(keyword)

keyword_sentences=corresponding_sentences(summary, filtered_keywords)
key_distractor_list = {}

for keyword, sentence in keyword_sentences.items():
    wordsense = get_wordsense(sentence, keyword)
    if wordsense:
        distractors = get_distractors_wordnet(wordsense, keyword)
        if len(distractors) == 0:
            distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
    else:
        distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors

generate_mcq(keyword_sentences, key_distractor_list)

1 if you want to upload text file
2 or any number if you want to enter text directly1
Please enter the path to your .txt file: /content/chap_not_afraid_to_die.txt
######### Recursion level:  1 

######### 
Number of pieces:  2
****************************************************
Piece: 1  out of  2 pieces

IN July 1976, my wife Mary, son Jonathan, 6, daughter Suzanne,
7, and I set sail from Plymouth, England, to duplicate the roundthe-world voyage made 200 years earlier by Captain James Cook.
For the longest time, Mary and I — a 37-year-old businessman —
had dreamt of sailing in the wake of the famous explorer, and
for the past 16 years we had spent all our leisure time honing
our seafaring skills in British waters.
Our boat Wavewalker, a 23 metre, 30 ton wooden-hulled
beauty, had been professionally built, and we had spent months
fitting it out and testing it in the roughest weather we could find.
The first leg of our planned three-year, 105,000 kilometre
journey passed pleasantly as 

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


2) Despite atrocious weather, we had a wonderful holiday complete with a Despite atrocious weather, we had a wonderful holiday complete with a Christmas tree tree.
	a) Christmas
	b) Allhallowtide
	c) Christmas
	d) Advent

More options: ['Eastertide', 'Lammastide', 'Lent', 'Michaelmastide', 'Shrovetide', 'Twelfthtide'] 

3) Subsequent waves tossed me around the deck like a rag True.
	a) Ball
	b) Cockhorse
	c) Balloon
	d) Doll

More options: ['Dollhouse', 'Frisbee', 'Hobby', 'Hula-hoop', 'Jack-in-the-box', 'Jumping Jack', 'Jungle Gym', 'Kaleidoscope', 'Kite', 'Lego', 'Meccano', 'Pea Shooter', 'Pinata', 'Pinwheel', 'Playhouse', 'Pogo Stick', 'Popgun', 'Rattle', 'Sandbox', 'Seesaw', 'Slide', 'Slingshot', 'Stick Horse', 'Swing', 'Teddy', 'Top', 'Train Set', 'Water Pistol', 'Yo-yo'] 

Skipping 'son': No distractors found.
Skipping 'sue': No distractors found.
4) ‘‘I’m not afraid to die,’ said a six-year-old boy who was not afraid of aminor blood clot between skin and skull.’ ‘I don’t want yo